## 1: Elabore uma solução para identificar o OUTPUT Grade do estudante.

### a) Faça o pré-processamento dos dados (limpeza, engenharia de variáveis, etc). Utilize pelo menos uma forma de redimensionamento de atributos (selecionando ou agregando) e utilize ambas as amostras para avaliar os resultados.

### b) Faça uma análise exploratória dos dados de saída, utilizando box plot e aplicando undersampling ou oversampling caso preciso.

### c) Utilize a biblioteca AutoML para fazer a previsão, com validação cruzada estratificada 10-fold e "random_state" fixo. Identifique o algoritmo que teve o melhor resultado e teste 3 variações de um dos seus hiperparâmetros. Avalie os resultados com matrizes de confusão e 2 métricas de sua preferência.

## 2: Utilizando a mesma base de dados da questão anterior:

### a) Agrupe os resultados através do método k-means, com k = 5 e k = 7.

### b) Agrupe os resultados através do método hierárquico, para 5 e 7 grupos e com 2 tipos diferentes de linkage.

### c) Compare os resultados das execuções anteriores e avalie uma métrica de avaliação para clusterização.

## 3: Quais são as vantagens e desvantagens de utilizar a biblioteca AutoML? Como você faria para usá-la, garantindo os melhores resultados possíveis?